# utils
> Utilities
- order: 12

In [1]:
#| default_exp utils

In [2]:
!which python

/Users/kobus/aishiftscheduler/.venv/bin/python


In [3]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
from collections import defaultdict
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
# import time
# import math
# from pprint import pprint
## !pip install -U "ray"
# import ray
# import json

from fastcore.basics import patch
import aishiftscheduler.config as cf
# import aishiftscheduler.policy as pol
# import aishiftscheduler.visualization as vis

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [6]:
#| export
def print_schedule_shifts(df, pars):
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  print(f"SCHEDULE SHIFTS:")
  print(f"===============")
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    print(f'\n************** {resName}:')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ts_1 = sched_list[0]
      dow_1 = sched_list[0].day_of_week
      print(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1:
          print(f"{(ts_1).strftime('%a %b %d %Hh%M')}\n")
          print(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        dow_1 = dow
        ts_1 = ts
      print(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}")
  print(f'\n{cf.CONTIGUOUS_REWARD=}')
  print(f'{pars.MAX_DAILY_SLOT_RUN=} ({pars.RESOLUTION}s)')
  print(f'{cf.TH_CumSlots_SPEC=}')
  print(f'{cf.TH_SickProb_SPEC=}')
  print(f'{cf.TH_CumMerits_SPEC=}')
  print(f'{cf.TH_ContSlots_SPEC=}')
  print(f'{cf.TH_Select_SPEC=}')

In [7]:
#| export
def gap_minutes(resolution):
  if resolution == 'QUARTER_HOUR':
    return 15
  elif resolution == 'HOUR':
    return 60
  elif resolution == 'BLOCK_8_HOUR':
    return 480
  else:
    print(f'ERROR: Invalid RESOLUTION: {cf.RESOLUTION}')

In [8]:
#| export
def print_schedule_slots(df, pars):
  gap_mins = gap_minutes(pars.RESOLUTION)
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  print(f"SCHEDULE SLOTS:")
  print(f"===============")
  n_gaps = 0
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    print(f'\n************** {resName}:')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ## print each slot and indicate gaps
      ts_1 = sched_list[0]
      dow_1 = -1
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1: print('')
        if (int((ts.to_datetime64() - ts_1.to_datetime64())/(1e9*60)) > gap_mins)\
          and (dow == dow_1):
          n_gaps += 1
          print(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP")
        else:
          print(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        dow_1 = dow
        ts_1 = ts
  print(f'\nTOTAL NUMBER OF GAPS FOR ALL RESOURCES: {n_gaps}')
  print(f'{cf.CONTIGUOUS_REWARD=}')
  print(f'{pars.MAX_DAILY_SLOT_RUN=} ({pars.RESOLUTION}s)')
  print(f'{cf.TH_CumSlots_SPEC=}')
  print(f'{cf.TH_SickProb_SPEC=}')
  print(f'{cf.TH_CumMerits_SPEC=}')
  print(f'{cf.TH_ContSlots_SPEC=}')
  print(f'{cf.TH_Select_SPEC=}')

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()